In [2]:
from langgraph.graph import StateGraph,START,END
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from typing import TypedDict
from langgraph.checkpoint.memory import InMemorySaver
import os
import operator

load_dotenv()

True

In [5]:
groq_api_key=os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")

In [4]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [6]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [7]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [8]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why was the pizza in a bad mood? \n\nBecause it was feeling a little crusty.',
 'explanation': 'This joke relies on a play on words. It starts with a question about why the pizza is in a bad mood, which sets up the expectation that the answer will be related to its emotional state. However, the punchline "it was feeling a little crusty" is a pun.\n\nIn this case, "crusty" has a double meaning. On one hand, it can describe something that is emotionally irritable or in a bad mood. On the other hand, a "crusty" can also refer to the crust of a pizza, which is a literal part of the food. The joke is funny because it takes the second meaning of "crusty" (the pizza crust) and applies it to the first meaning (feeling irritable), creating a clever and unexpected connection between the setup and the punchline.'}